Notebook de préparation au modèle d'estimation

Nettoyage du dataset de biens immobiliers 2020
- Retirer les nuls
- Ploter
- Valeurs abhérantes
- Isolation forest

Split du dataset en Départements

Etalonnage d'un modèle de regression linéaire

Test du modèle

In [2]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import re
import numpy as np
import seaborn as sn
import sklearn
import matplotlib.pyplot as plt
from classes.credentials import Credentials as cr
from classes.prediction import ImmothepPrediction
import os
from classes.split import Splitter
from sklearn.ensemble import IsolationForest

splitter = Splitter()
predictor = ImmothepPrediction()

In [12]:
valeurs2020 = pd.read_csv(cr.PATH+'2020.txt', sep='|', usecols=['Nature mutation', 'Code postal','Nombre pieces principales', 'Surface terrain', 'Surface Carrez du 1er lot', 'Surface Carrez du 2eme lot', 'Surface Carrez du 3eme lot', 'Surface Carrez du 4eme lot', 'Surface Carrez du 5eme lot', 'Valeur fonciere', 'Type local', 'Nombre de lots', "Surface reelle bati"], encoding='utf-8')

valeurs2020 = valeurs2020.dropna(subset = ['Type local', 'Nombre de lots', 'Nombre pieces principales', "Nature mutation", "Code postal", "Valeur fonciere"])

p = re.compile('(appartement|maison){1}\|{1}[0-9]*(.|,)?[0-9]*\|{1}[0-9]*(.|,)?[0-9]*\|{1}[0-9]*(.|,)?[0-9]*\|{1}[0-9]*(.|,)?[0-9]*\|{1}[0-9]*(.|,)?[0-9]*')

for line in valeurs2020:
    print(line)
    #for ligne in fichier:
        #print(ligne)
        #m = p.match(ligne)
        #print(m)

C:\Users\utilisateur\anaconda3\envs\data-ia\lib\site-packages\IPython\core\interactiveshell.py:3169: DtypeWarning: Columns (31,33) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


Nature mutation
Valeur fonciere
Code postal
Surface Carrez du 1er lot
Surface Carrez du 2eme lot
Surface Carrez du 3eme lot
Surface Carrez du 4eme lot
Surface Carrez du 5eme lot
Nombre de lots
Type local
Surface reelle bati
Nombre pieces principales
Surface terrain


In [53]:
valeurs2020['Surface terrain'] = valeurs2020['Surface terrain'].fillna(0) 

valeurs2020 = valeurs2020[[
    'Type local',
    'Code postal', 
    'Nombre pieces principales', 
    'Surface terrain', 
    'Surface reelle bati',
    'Valeur fonciere'
    ]][valeurs2020['Type local'].str.contains("Appartement|Maison", regex=True)][valeurs2020['Nature mutation'].str.contains("Vente", regex=True)][valeurs2020['Nombre de lots']<6][valeurs2020['Nombre pieces principales']>0][valeurs2020['Surface reelle bati']>=8]

valeurs2020.head()

C:\Users\UTILIS~1\AppData\Local\Temp/ipykernel_7384/209343670.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  valeurs2020 = valeurs2020[[
C:\Users\UTILIS~1\AppData\Local\Temp/ipykernel_7384/209343670.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  valeurs2020 = valeurs2020[[
C:\Users\UTILIS~1\AppData\Local\Temp/ipykernel_7384/209343670.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  valeurs2020 = valeurs2020[[
C:\Users\UTILIS~1\AppData\Local\Temp/ipykernel_7384/209343670.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  valeurs2020 = valeurs2020[[


,Type local,Code postal,Nombre pieces principales,Surface terrain,Surface reelle bati,Valeur fonciere
11,Maison,1270.0,2.0,381.0,35.0,"72000,00"
13,Maison,1000.0,4.0,525.0,75.0,"180300,00"
14,Appartement,1000.0,1.0,0.0,32.0,"54800,00"
16,Maison,1000.0,7.0,1267.0,201.0,"350750,00"
17,Maison,1000.0,7.0,1497.0,201.0,"350750,00"


In [54]:
valeurs2020[['Valeur fonciere']] = valeurs2020[['Valeur fonciere']].replace(',', '.', regex=True)

valeurs2020[['Valeur fonciere']].astype('float')

,Valeur fonciere
11,72000.0
13,180300.0
14,54800.0
16,350750.0
17,350750.0
...,...
2459552,1937500.0
2459553,1937500.0
2459554,1937500.0
2459555,1937500.0


In [55]:
valeurs2020.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 915817 entries, 11 to 2459557
Data columns (total 6 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   Type local                 915817 non-null  object 
 1   Code postal                915817 non-null  float64
 2   Nombre pieces principales  915817 non-null  float64
 3   Surface terrain            915817 non-null  float64
 4   Surface reelle bati        915817 non-null  float64
 5   Valeur fonciere            915817 non-null  object 
dtypes: float64(4), object(2)
memory usage: 48.9+ MB


In [77]:
nancount = valeurs2020["Valeur fonciere"].isna()
print ("Il y a ", nancount.sum(), " valeurs nulles dans la colonne \'Valeur Foncière\'.")
valeurs2020 = valeurs2020.dropna(subset = ['Valeur fonciere'])
nancount = valeurs2020["Valeur fonciere"].isna()
print ("Il y a ", nancount.sum(), " valeurs nulles dans la colonne \'Valeur Foncière\'.")

nancount = valeurs2020["Code postal"].isna()
print ("Il y a ", nancount.sum(), " valeurs nulles dans la colonne \'Code postal\'.")
valeurs2020 = valeurs2020.dropna(subset = ['Code postal'])
nancount = valeurs2020["Code postal"].isna()
print ("Il y a ", nancount.sum(), " valeurs nulles dans la colonne \'Code postal\'.")

Il y a  0  valeurs nulles dans la colonne 'Valeur Foncière'.
Il y a  0  valeurs nulles dans la colonne 'Valeur Foncière'.
Il y a  0  valeurs nulles dans la colonne 'Code postal'.
Il y a  0  valeurs nulles dans la colonne 'Code postal'.


In [63]:
maisons2020=valeurs2020[valeurs2020['Type local'].str.contains("Maison", regex=True)].reset_index(drop=True)

appart2020=valeurs2020[valeurs2020['Type local'].str.contains("Appartement", regex=True)].reset_index(drop=True)

In [64]:
maisons2020.to_csv(cr.CURATED_LOCAL_PATH + 'filteredmaisons2020.csv', index=False)
appart2020.to_csv(cr.CURATED_LOCAL_PATH + 'filteredappart2020.csv', index=False)

In [65]:
maisons2020 = pd.read_csv(cr.CURATED_LOCAL_PATH + 'filteredmaisons2020.csv', sep=",")
appart2020 = pd.read_csv(cr.CURATED_LOCAL_PATH + 'filteredappart2020.csv', sep=",")

In [67]:
# Split des données par type de logement et par département
splitter.split_datas('filteredmaisons2020.csv', 'Code postal', 'CPMaisons')
splitter.split_datas('filteredappart2020.csv', 'Code postal', 'CPAppart')

In [76]:
valeursmaisons63 = pd.read_csv(cr.CURATED_LOCAL_PATH + 'CPMaisons/63.csv', sep='|', encoding="utf-8")

model = IsolationForest(valeursmaisons63, n_estimators=50, max_samples='auto', contamination='auto', max_features=1.0, bootstrap=False, n_jobs=None, random_state=None, verbose=0, warm_start=False)
model.fit(valeursmaisons63[['Valeur fonciere']])

C:\Users\utilisateur\anaconda3\envs\data-ia\lib\site-packages\sklearn\utils\validation.py:67: FutureWarning: Pass n_estimators=    Type local  Code postal  Nombre pieces principales  Surface terrain  \
0       Maison      63950.0                        3.0            580.0   
1       Maison      63950.0                        3.0            400.0   
2       Maison      63950.0                        4.0              0.0   
3       Maison      63950.0                        2.0           1670.0   
4       Maison      63950.0                        2.0            500.0   
..         ...          ...                        ...              ...   
258     Maison      63200.0                        5.0            922.0   
259     Maison      63200.0                        4.0            659.0   
260     Maison      63200.0                        5.0            236.0   
261     Maison      63200.0                        3.0             60.0   
262     Maison      63200.0                     

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').